In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [10]:
df1=pd.read_excel('./online_retail_II.xlsx', sheet_name='Year 2009-2010')
df2=pd.read_excel('./online_retail_II.xlsx', sheet_name='Year 2010-2011')

In [11]:
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [12]:
df2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [13]:
print("Shape df1:", df1.shape)
print("Shape df2:", df2.shape)

Shape df1: (525461, 8)
Shape df2: (541910, 8)


In [14]:
df1.isna().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [15]:
df2.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [16]:
df1.dropna(inplace=True)
df2.dropna(inplace=True)

In [17]:
print("Shape df1:", df1.shape)
print("Shape df2:", df2.shape)

Shape df1: (417534, 8)
Shape df2: (406830, 8)


In [22]:
df1=df1[~df1['Invoice'].str.contains("C", na=False)]
df2=df2[~df2['Invoice'].str.contains("C", na=False)]

In [23]:
print("Shape df1:", df1.shape)
print("Shape df2:", df2.shape)

Shape df1: (407695, 8)
Shape df2: (397925, 8)


### RFM Analysis

In [26]:
print("Min Invoice Date in the year 2009-2010:", df1.InvoiceDate.min())
print("Max Invoice Date in the year 2009-2010:", df1.InvoiceDate.max())

Min Invoice Date in the year 2009-2010: 2009-12-01 07:45:00
Max Invoice Date in the year 2009-2010: 2010-12-09 20:01:00


In [29]:
print("Min Invoice Date in the year 2010-2011:", df2.InvoiceDate.min())
print("Max Invoice Date in the year 2010-2011:", df2.InvoiceDate.max())

Min Invoice Date in the year 2010-2011: 2010-12-01 08:26:00
Max Invoice Date in the year 2010-2011: 2011-12-09 12:50:00


In [45]:
#Year 2009-2010
recency1=(dt.datetime(2010, 12, 9)-df1.groupby(['Customer ID']).agg({"InvoiceDate": "max"})).rename(columns={'InvoiceDate': 'Recency'})
#Year 2010-2011
recency2=(dt.datetime(2011, 12, 9)-df1.groupby(['Customer ID']).agg({"InvoiceDate": "max"})).rename(columns={'InvoiceDate': 'Recency'})

recency1["Recency"] = recency1["Recency"].apply(lambda x: x.days)
recency2["Recency"] = recency2["Recency"].apply(lambda x: x.days)

recency1.head()

,Recency
Customer ID,
12346.0,163
12347.0,1
12348.0,72
12349.0,41
12351.0,9


In [46]:
freq1 = df1.groupby("Customer ID").agg({"InvoiceDate":"nunique"}).rename(columns={"InvoiceDate": "Frequency"})
freq2 = df2.groupby("Customer ID").agg({"InvoiceDate":"nunique"}).rename(columns={"InvoiceDate": "Frequency"})

freq2

,Frequency
Customer ID,
12346.0,1
12347.0,7
12348.0,4
12349.0,1
12350.0,1
...,...
18280.0,1
18281.0,1
18282.0,2


In [47]:
df1["TotalPrice"] = df1["Quantity"] * df1["Price"]
df2["TotalPrice"] = df2["Quantity"] * df2["Price"]

monetary1 = df1.groupby("Customer ID").agg({"TotalPrice":"sum"}).rename(columns={"TotalPrice":"Monetary"})
monetary2 = df2.groupby("Customer ID").agg({"TotalPrice":"sum"}).rename(columns={"TotalPrice":"Monetary"})

monetary2.head()

,Monetary
Customer ID,
12346.0,77183.60
12347.0,4310.00
12348.0,1797.24
12349.0,1757.55
12350.0,334.40


In [48]:
rfm1 = pd.concat([recency1, freq1, monetary1],  axis=1)
rfm2 = pd.concat([recency2, freq2, monetary2],  axis=1)

rfm2.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,528.0,1.0,77183.60
12347.0,366.0,7.0,4310.00
12348.0,437.0,4.0,1797.24
12349.0,406.0,1.0,1757.55
12351.0,374.0,NaN,NaN


In [51]:
rfm1["RecencyScore"] = pd.qcut(rfm1["Recency"], 5, labels = [5, 4 , 3, 2, 1])
rfm2["RecencyScore"] = pd.qcut(rfm2["Recency"], 5, labels = [5, 4 , 3, 2, 1])

rfm1["FrequencyScore"]= pd.qcut(rfm1["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])
rfm2["FrequencyScore"]= pd.qcut(rfm2["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])

rfm1["MonetaryScore"] = pd.qcut(rfm1['Monetary'], 5, labels = [1, 2, 3, 4, 5])
rfm2["MonetaryScore"] = pd.qcut(rfm2['Monetary'], 5, labels = [1, 2, 3, 4, 5])

rfm2.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346.0,528.0,1.0,77183.60,2,1,5,215
12347.0,366.0,7.0,4310.00,5,5,5,555
12348.0,437.0,4.0,1797.24,2,4,4,244
12349.0,406.0,1.0,1757.55,3,1,4,314
12351.0,374.0,NaN,NaN,5,NaN,NaN,5nannan


In [52]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

rfm1['Segment'] = rfm1['RecencyScore'].astype(str) + rfm1['FrequencyScore'].astype(str)
rfm1['Segment'] = rfm1['Segment'].replace(seg_map, regex=True)

rfm2['Segment'] = rfm2['RecencyScore'].astype(str) + rfm2['FrequencyScore'].astype(str)
rfm2['Segment'] = rfm2['Segment'].replace(seg_map, regex=True)

rfm2.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,528.0,1.0,77183.60,2,1,5,215,Hibernating
12347.0,366.0,7.0,4310.00,5,5,5,555,Champions
12348.0,437.0,4.0,1797.24,2,4,4,244,At Risk
12349.0,406.0,1.0,1757.55,3,1,4,314,About to Sleep
12351.0,374.0,NaN,NaN,5,NaN,NaN,5nannan,5nan
